In [2]:
import boto3
import os

from dotenv import load_dotenv
from langchain_aws import BedrockLLM
from langchain.agents import tool
from langchain_core.prompts import PromptTemplate

load_dotenv()

True

In [3]:
# 1. load LLM
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name = os.getenv('AWS_DEFAULT_REGION'),
    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
)

llm = BedrockLLM(model_id='anthropic.claude-v2:1', model_kwargs={"temperature": 0}, client=bedrock_client)


In [4]:
# 2. define the tools
from datetime import datetime

@tool
def date_calculator(start_date,end_date)->int:
    """Return the total days between start_date and end_date"""
    start_date_object = datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date_object = datetime.strptime(end_date, '%Y-%m-%d').date()
    duration = end_date_object - start_date_object

    return duration.days



In [5]:
date_calculator.invoke({"start_date":'2024-01-03',"end_date":"2024-01-05"})

2

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert on data analysis. You need to analyze dataframe and answer the question base on the dataframe.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)